In [ ]:
import re

import ndjson

with open("input.ndjson") as f:
    data = ndjson.load(f)


def get_key_from_url(url):
    match = re.match("^https://mattsbeta.s3.us-west-1.amazonaws.com/(.*)", url)
    if match is None:
        return
    (key,) = match.groups()
    return key


def add_cover_photo_key(row):
    cover_photo_url = row.get("coverPhotoUrl")
    if cover_photo_url is None:
        return
    key = get_key_from_url(cover_photo_url)
    if key is not None:
        row["coverPhotoS3Key"] = key


def add_report_photo_keys(row):
    report = row.get("report")
    if report is None:
        return
    for block in report:
        if block["_type"] != "photo":
            continue
        url = block.get("url")
        if url is None:
            continue
        key = get_key_from_url(url)
        if key is not None:
            block["s3Key"] = key


for row in data:
    if row["_type"] == "trip":
        add_cover_photo_key(row)
        add_report_photo_keys(row)

with open("output.ndjson", "w") as f:
    ndjson.dump(data, f)